# Test validity step 3 : $P(S_c | ...) \propto s_c^t S_c^{-1} s_c + ln|S_c|$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import astropy.io.fits as fits
import camb

In [ ]:
nstokes = 3
nside = 64
lmax = nside*2
